In [10]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType,DecimalType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [2]:
spark = (SparkSession
 .builder
 .appName("State_Deviation")
 .getOrCreate())

22/08/27 08:09:20 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/27 08:09:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/27 08:09:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/27 08:09:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/27 08:09:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/27 08:09:26 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [44]:
schema = StructType([
             StructField('State', StringType(), True),
             StructField('Salary_Deviation', StringType(), True)
         ])

In [104]:
df =  spark.read.option('header','True').schema(schema).csv('German_Salaries_Deviation_By_State.csv')

In [105]:
df = df.withColumn('Salary_Deviation',regexp_replace(df.Salary_Deviation,pattern='[!,% ]', replacement=''))

In [106]:
df.show()

+--------------------+----------------+
|               State|Salary_Deviation|
+--------------------+----------------+
|   Baden-Württemberg|             +79|
|               Hesse|             +72|
|             Hamburg|             +55|
|             Bavaria|             +44|
|North Rhine-Westp...|             +11|
|Rhineland-Palatinate|             -14|
|              Bremen|             -29|
|            Saarland|             -44|
|              Berlin|             -57|
|        Lower Saxony|             -68|
|  Schleswig-Holstein|             -98|
|           Thuringia|            -170|
|              Saxony|            -185|
|         Brandenburg|            -188|
|       Saxony-Anhalt|            -195|
|Mecklenburg-Vorpo...|            -220|
+--------------------+----------------+



In [107]:
df = df.withColumn("Salary_Deviation",df['Salary_Deviation'].cast('double')/10)


In [108]:
df.show()

+--------------------+----------------+
|               State|Salary_Deviation|
+--------------------+----------------+
|   Baden-Württemberg|             7.9|
|               Hesse|             7.2|
|             Hamburg|             5.5|
|             Bavaria|             4.4|
|North Rhine-Westp...|             1.1|
|Rhineland-Palatinate|            -1.4|
|              Bremen|            -2.9|
|            Saarland|            -4.4|
|              Berlin|            -5.7|
|        Lower Saxony|            -6.8|
|  Schleswig-Holstein|            -9.8|
|           Thuringia|           -17.0|
|              Saxony|           -18.5|
|         Brandenburg|           -18.8|
|       Saxony-Anhalt|           -19.5|
|Mecklenburg-Vorpo...|           -22.0|
+--------------------+----------------+



In [109]:
window = Window.orderBy('State')
df = df.withColumn('Id',row_number().over(window))

df.show(truncate=False)

22/08/27 11:07:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 11:07:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------+----------------+---+
|State                 |Salary_Deviation|Id |
+----------------------+----------------+---+
|Baden-Württemberg     |7.9             |1  |
|Bavaria               |4.4             |2  |
|Berlin                |-5.7            |3  |
|Brandenburg           |-18.8           |4  |
|Bremen                |-2.9            |5  |
|Hamburg               |5.5             |6  |
|Hesse                 |7.2             |7  |
|Lower Saxony          |-6.8            |8  |
|Mecklenburg-Vorpommern|-22.0           |9  |
|North Rhine-Westphalia|1.1             |10 |
|Rhineland-Palatinate  |-1.4            |11 |
|Saarland              |-4.4            |12 |
|Saxony                |-18.5           |13 |
|Saxony-Anhalt         |-19.5           |14 |
|Schleswig-Holstein    |-9.8            |15 |
|Thuringia             |-17.0           |16 |
+----------------------+----------------+---+



In [110]:
df.write.format('csv').option('header','true').mode('overwrite').save('State_Salary_Deviation')

22/08/27 11:07:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 11:07:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 11:07:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
